<a href="https://assets.datacamp.com/production/repositories/6312/datasets/e40061c2b58e9dbe22e94c4c96c18322676a3d40/omniglot_train.zip">omniglot_train</a> <br/>
<a href="https://assets.datacamp.com/production/repositories/6312/datasets/c48a507888bf774d66d7ab3ef77e37a8091179ec/omniglot_test.zip">omniglot_test</a> <br/>
<a href="https://assets.datacamp.com/production/repositories/6312/datasets/b11ec13ff18fb769e6d9370b2b53aaee3a08b501/electricity_consump.zip">electricity_consump</a> <br/>
<a href="https://assets.datacamp.com/production/repositories/6312/datasets/8a96624719a082926c7fad09e235f78a9f63656b/clouds.zip">clouds</a> <br/>
<a href="https://assets.datacamp.com/production/repositories/6312/datasets/f6a56ec01791d157db6c60ad10dc961e5469433f/water_potability.zip">water_potability</a> <br/>

In [16]:
import numpy as np
import pandas as pd

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchmetrics import Accuracy

## PyTorch Dataset

#### Time to refresh your PyTorch Datasets knowledge!

##### Before model training can commence, you need to load the data and pass it to the model in the right format. In PyTorch, this is handled by Datasets and DataLoaders. Let's start with building a PyTorch Dataset for our water potability data.

##### In this exercise, you will define a class called WaterDataset to load the data from a CSV file. To do this, you will need to implement the three methods which PyTorch expects a Dataset to have:

##### -    .__init__() to load the data,
##### -    .__len__() to return data size,
##### -    .__getitem()__ to extract features and label for a single sample.

##### The following imports that you need have already been done for you:

<code>
import pandas as pd
from torch.utils.data import Dataset
</code>

### Instructions 1/3
-    In the .__init__() method, load the data from csv_path to a pandas DataFrame and assign it to df.
-    Convert df to a NumPy array and assign the result to self.data.

In [17]:
class WaterDataset(Dataset):
    def __init__(self, csv_path):
        super().__init__()
        # Load data to pandas DataFrame
        df = pd.read_csv(csv_path)
        # Convert data to a NumPy array and assign to self.data
        self.data = df.to_numpy()

### Instructions 2/3
-    Implement the .__len__() method to return the number of data samples.

In [18]:
class WaterDataset(Dataset):
    def __init__(self, csv_path):
        super().__init__()
        # Load data to pandas DataFrame
        df = pd.read_csv(csv_path)
        # Convert data to a NumPy array and assign to self.data
        self.data = df.to_numpy()
      
    # Implement __len__ to return the number of data samples
    def __len__(self):
        return self.data.shape[0]

### Instructions 3/3
-    In the .__getitem__() method, get the label by slicing self.data to extract its last column for the index idx, similarly to how it's done for the features.

In [19]:
class WaterDataset(Dataset):
    def __init__(self, csv_path):
        super().__init__()
        # Load data to pandas DataFrame
        df = pd.read_csv(csv_path)
        # Convert data to a NumPy array and assign to self.data
        self.data = df.to_numpy()
        
    # Implement __len__ to return the number of data samples
    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, idx):
        features = self.data[idx, :-1]
        # Assign last data column to label
        label = self.data[idx, -1]
        return features, label

## PyTorch DataLoader

##### Good job defining the Dataset class! The WaterDataset you just created is now available for you to use.

##### The next step in preparing the training data is to set up a DataLoader. A PyTorch DataLoader can be created from a Dataset to load data, split it into batches, and perform transformations on the data if desired. Then, it yields a data sample ready for training.

##### In this exercise, you will build a DataLoader based on the WaterDataset. The DataLoader class you will need has already been imported for you from torch.utils.data. Let's get to it!

### Instructions
-    Create an instance of WaterDataset from water_train.csv, assigning it to dataset_train.
-    Create dataloader_train based on dataset_train, using a batch size of two and shuffling the samples.
-    Get a batch of features and labels from the DataLoader and print them.

In [20]:
# Create an instance of the WaterDataset
#dataset_train = WaterDataset("water_train.csv")
dataset_train = WaterDataset("data/water_train.csv")

# Create a DataLoader based on dataset_train
dataloader_train = DataLoader(
    dataset_train,
    batch_size=2,
    shuffle=True,
)

# Get a batch of features and labels
features, labels = next(iter(dataloader_train))
print(features, labels)

tensor([[0.5727, 0.6685, 0.4684, 0.4155, 0.4150, 0.5354, 0.6606, 0.4309, 0.2388],
        [0.4076, 0.3461, 0.2807, 0.6874, 0.4220, 0.6317, 0.3898, 0.3783, 0.7135]],
       dtype=torch.float64) tensor([0., 0.], dtype=torch.float64)


## PyTorch Model

##### You will use the OOP approach to define the model architecture. Recall that this requires setting up a model class and defining two methods inside it:

##### -    .__init__(), in which you define the layers you want to use;
##### -    forward(), in which you define what happens to the model inputs once it receives them; this is where you pass inputs through pre-defined layers.

##### Let's build a model with three linear layers and ReLU activations. After the last linear layer, you need a sigmoid activation instead, which is well-suited for binary classification tasks like our water potability prediction problem. Here's the model defined using nn.Sequential(), which you may be more familiar with:

<code>
net = nn.Sequential(
  nn.Linear(9, 16),
  nn.ReLU(),
  nn.Linear(16, 8),
  nn.ReLU(),
  nn.Linear(8, 1),
  nn.Sigmoid(),
)
</code>

###### Let's rewrite this model as a class!

### Instructions
-    In the .__init__() method, define the three linear layers with dimensions corresponding to the model definition provided and assign them to self.fc1, self.fc2, and self.fc3, respectively.
-    In the forward() method, pass the model input x through all the layers, remembering to add activations on top of them, similarly how it's already done for the first layer.

In [34]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Define the three linear layers
        self.fc1 = nn.Linear(9, 16).double()
        self.fc2 = nn.Linear(16, 8).double()
        self.fc3 = nn.Linear(8, 1).double()
    
    def forward(self, x):
        # Pass x through linear layers adding activations
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = nn.functional.sigmoid(self.fc3(x))
        return x

## Optimizers

##### It's time to explore the different optimizers that you can use for training your model.

##### A custom function called train_model(optimizer, net, num_epochs) has been defined for you. It takes the optimizer, the model, and the number of epochs as inputs, runs the training loops, and prints the training loss at the end.

##### Let's use train_model() to run a few short trainings with different optimizers and compare the results!

### Instructions 1/3
-    Define the optimizer as Stochastic Gradient Descent.

In [35]:
import torch.nn as nn
import torch.nn.functional as F

## 
def train_model(optimizer, net, num_epochs):
    criterion = nn.BCELoss()
    for epoch in range(num_epochs):
        running_loss = 0.
        for features, labels in dataloader_train:
            optimizer.zero_grad()
            outputs = net(features)
            loss = criterion(outputs, labels.view(-1, 1))
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
    train_loss = running_loss / len(dataloader_train)
    print(f"Training loss after {num_epochs} epochs: {train_loss}")

In [36]:
import torch.optim as optim

net = Net()

# Define the SGD optimizer
optimizer = optim.SGD(net.parameters(), lr=0.001)

train_model(
    optimizer=optimizer,
    net=net,
    num_epochs=10,
)

Training loss after 10 epochs: 0.6735866321318518


### Instructions 2/3
-    Define the optimizer as Root Mean Square Propagation (RMSprop), passing the model's parameters as its first argument.

In [37]:
import torch.optim as optim

net = Net()

# Define the RMSprop optimizer
optimizer = optim.RMSprop(net.parameters(), lr=0.001)

train_model(
    optimizer=optimizer,
    net=net,
    num_epochs=10,
)

Training loss after 10 epochs: 0.6716230012652026


### Instructions 3/3
-    Define the optimizer as Adaptive Moments Estimation (Adam), setting the learning rate to 0.001.

In [38]:
import torch.optim as optim

net = Net()

# Define the Adam optimizer
optimizer = optim.Adagrad(net.parameters(), lr=0.01)

train_model(
    optimizer=optimizer,
    net=net,
    num_epochs=10,
)

Training loss after 10 epochs: 0.6734744689608814


## Model evaluation

##### With the training loop sorted out, you have trained the model for 1000 epochs, and it is available to you as net. You have also set up a test_dataloader in exactly the same way as you did with train_dataloader before—just reading the data from the test rather than the train directory.

##### You can now evaluate the model on test data. To do this, you will need to write the evaluation loop to iterate over the batches of test data, get the model's predictions for each batch, and calculate the accuracy score for it. Let's do it!

### Instructions
-    Set up the evaluation metric as Accuracy for binary classification and assign it to acc.
-    For each batch of test data, get the model's outputs and assign them to outputs.
-    After the loop, compute the total test accuracy and assign it to test_accuracy.

In [43]:
# ! pip install torchmetrics
dataset_test = WaterDataset("data/water_test.csv")

# Create a DataLoader based on dataset_train
dataloader_test = DataLoader(
    dataset_test,
    batch_size=2,
    shuffle=True,
)

In [44]:
import torch
from torchmetrics import Accuracy

# Set up binary accuracy metric
acc = Accuracy(task="binary")

net.eval()
with torch.no_grad():
    for features, labels in dataloader_test:
        # Get predicted probabilities for test data batch
        outputs = net(features)
        preds = (outputs >= 0.5).float()
        acc(preds, labels.view(-1, 1))

# Compute total test accuracy
test_accuracy = acc.compute()
print(f"Test accuracy: {test_accuracy}")

Test accuracy: 0.5904572606086731


## Initialization and activation

##### The problems of unstable (vanishing or exploding) gradients are a challenge that often arises in training deep neural networks. In this and the following exercises, you will expand the model architecture that you built for the water potability classification task to make it more immune to those problems.

##### As a first step, you'll improve the weights initialization by using He (Kaiming) initialization strategy. To do so, you will need to call the proper initializer from the torch.nn.init module, which has been imported for you as init. Next, you will update the activations functions from the default ReLU to the often better ELU.

### Instructions
-    Call the He (Kaiming) initializer on the weight attribute of the second layer, fc2, similarly to how it's done for fc1.
-    Call the He (Kaiming) initializer on the weight attribute of the third layer, fc3, accounting for the different activation function used in the final layer.
-    Update the activation functions in the forward() method from relu to elu.

In [45]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(9, 16)
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, 1)
        
        # Apply He initialization
        init.kaiming_uniform_(self.fc1.weight)
        init.kaiming_uniform_(self.fc2.weight)
        init.kaiming_uniform_(self.fc3.weight, nonlinearity="sigmoid")

    def forward(self, x):
        # Update ReLU activation to ELU
        x = nn.functional.elu(self.fc1(x))
        x = nn.functional.elu(self.fc2(x))
        x = nn.functional.sigmoid(self.fc3(x))
        return x

## Batch Normalization

##### As a final improvement to the model architecture, let's add the batch normalization layer after each of the two linear layers. The batch norm trick tends to accelerate training convergence and protects the model from vanishing and exploding gradients issues.

##### Both torch.nn and torch.nn.init have already been imported for you as nn and init, respectively. Once you implement the change in the model architecture, be ready to answer a short question on how batch normalization works!

### Instructions 1/3
-    Add two BatchNorm1d layers assigning them to self.bn1 and self.bn2.

In [46]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(9, 16)
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, 1)
        # Add two batch normalization layers
        self.bn1 = nn.BatchNorm1d(16)
        self.bn2 = nn.BatchNorm1d(8)
        
        init.kaiming_uniform_(self.fc1.weight)
        init.kaiming_uniform_(self.fc2.weight)
        init.kaiming_uniform_(self.fc3.weight, nonlinearity="sigmoid")

### Instructions 2/3
-    In the forward() method, pass x through the second set of layers: the linear layer, the batch norm layer, and the activations, similarly to how it's done for the first set of layers.

In [47]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(9, 16)
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, 1)
        # Add two batch normalization layers
        self.bn1 = nn.BatchNorm1d(16)
        self.bn2 = nn.BatchNorm1d(8)
        
        init.kaiming_uniform_(self.fc1.weight)
        init.kaiming_uniform_(self.fc2.weight)
        init.kaiming_uniform_(self.fc3.weight, nonlinearity="sigmoid") 
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x)
        x = nn.functional.elu(x)

        # Pass x through the second set of layers
        x = self.fc2(x)
        x = self.bn2(x)
        x = nn.functional.elu(x)

        x = nn.functional.sigmoid(self.fc3(x))
        return x